In [1]:
import nltk
import random
import os
import json
from nltk.stem import  PorterStemmer
import random
import pickle

In [2]:
def load_dataset():  
    dataset=[]
    for root, dirs, files in os.walk('../Dataset'):
        for directory in dirs:
            for root_f, dirs_f, files_f in os.walk(os.path.join(root,directory)):
                for each_file in files_f:
                    with open(os.path.join(root,directory)+'/'+each_file) as f:
                        data_temp=json.load(f)    
                    dataset+=[(sentence,directory) for sentence in data_temp]
    return dataset

In [3]:
#initialising porterstemmer
ps=PorterStemmer()

In [4]:
def to_unigram(data):
    dataset=[]
    dataset+=[([items for items in sentence.split()],aspect) for sentence,aspect in data]    
    return dataset    

In [5]:
def get_words(x):    #returns all the words in the  data set
    vocab = []
    for sentence,polarity in x:  
        vocab.extend(sentence.split())
    return vocab

In [6]:
#execution
labeled_data=load_dataset()    #loading labeled sentences
random.shuffle(labeled_data)

vocab=set(get_words(labeled_data)) #getting words 
vocab=[ps.stem(word) for word in vocab]   #stemming
#with open('vocab_unigram','w') as f:
#    json.dump(vocab,f)
labeled_data=to_unigram(labeled_data)    #converting to unigram

In [7]:
#bigrams
print labeled_data[100]

([u'enjoy', u'almost', u'everyth', u'els', u'job', u'pay'], 'Aspect2')


In [8]:
print len(labeled_data)

201378


In [9]:
print len(vocab)
print vocab[:100]

6544
[u'foul', u'interchang', u'four', u'payoff', u'authorit', u'scold', u'lore', u'lord', u'digit', u'pigment', u'yellow', u'delv', u'politician', u'disturb', u'prize', u'wooden', u'showca', u'habea', u'charter', u'erron', u'nigh', u'rustl', u'miller', u'histor', u'second', u'sooth', u'tether', u'dialogu', u'ruthless', u'thunder', u'specialist', u'aver', u'intellectu', u'crouch', u'avert', u'accu', u'herd', u'gamut', u'china', u'cult', u'deterior', u'militari', u'k', u'unforgett', u'fumbl', u'golden', u'brought', u'sterl', u'stern', u'unit', u'spoke', u'overshadow', u'music', u'telegraph', u'passport', u'strike', u'expiat', u'paperwork', u'relay', u'relax', u'relat', u'notic', u'hurt', u'glass', u'exc', u'holi', u'outskirt', u'midst', u'hold', u'accid', u'blade', u'conceptu', u'sweeter', u'etiquett', u'household', u'virtu', u'cautiou', u'caution', u'want', u'gargantuan', u'shuffl', u'unpaid', u'travel', u'cutback', u'hot', u'hop', u'turk', u'perspect', u'hoy', u'diagram', u'wrong', u'

In [10]:
def extract_features(document):       #features are bag of words. document is a list of words of a sentence 
    features = {}
    for word in vocab:
        features['contains(%s)' % word] = (word in document)
    return features        

In [11]:
train_test = nltk.classify.apply_features(extract_features, labeled_data)

In [12]:
#dividing into training and test data
train_set=train_test[:-10000]
test_set=train_test[-10000:]

In [19]:
#classifier = nltk.NaiveBayesClassifier.train(train_set)
#with open('naive_bayes_unigram_model','w') as f:
#    pickle.dump(classifier,f)

In [ ]:
#accuracy=nltk.classify.accuracy(classifier, test_set)
#with open('accuracy','w') as f:
#    json.dump(accuracy,f)

In [15]:
#load the model
with open('naive_bayes_unigram_model') as f:
    classifier=pickle.load(f)

In [40]:
#sample
sen='work opportun is bore '
classifier.classify(extract_features(sen.split()))

'Aspect1'

In [48]:
#calculating the probability
dist = classifier.prob_classify(extract_features(sen.split()))
polarity=0
for label in dist.samples():
    print("%s: %f" % (label, dist.prob(label)))
    polarity=max(polarity,dist.prob(label))
print polarity

Aspect7: 0.006441
Aspect6: 0.232167
Aspect5: 0.000175
Aspect4: 0.078496
Aspect3: 0.024981
Aspect2: 0.019896
Aspect1: 0.637844
0.637843926224
